In [2]:
from pyspark.sql import SparkSession
import pyspark as ps
import pyspark.sql.functions as F
from pyspark.sql.functions import col,lit,when
import pandas as pd

In [3]:
spark = (SparkSession
         .builder
         .appName('python Spark SQL')
         .config('spark.some.config.option','some-value')
         .getOrCreate())


your 131072x1 screen size is bogus. expect trouble
24/06/20 19:54:15 WARN Utils: Your hostname, Artemisia-PC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/20 19:54:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 19:54:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_spark = spark.read.format('csv').option('header','true').load('../datasets/Cancer_Data.csv')

In [5]:
parquet_path = '/home/arteweyl/coderhouse/python/turma-58865/datasets/parquets'

In [6]:
df_spark.write.parquet(f'{parquet_path}/Cancer_data.parquet',mode='overwrite')

24/06/20 19:59:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/06/20 19:59:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, diagnosis, radius_mean, texture_mean, perimeter_mean, area_mean, smoothness_mean, compactness_mean, concavity_mean, concave points_mean, symmetry_mean, fractal_dimension_mean, radius_se, texture_se, perimeter_se, area_se, smoothness_se, compactness_se, concavity_se, concave points_se, symmetry_se, fractal_dimension_se, radius_worst, texture_worst, perimeter_worst, area_worst, smoothness_worst, compactness_worst, concavity_worst, concave points_worst, symmetry_worst, fractal_dimension_worst, 
 Schema: id, diagnosis, radius_mean, texture_mean, perimeter_mean, area_mean, smoothness_mean, compactness_mean, concavity_mean, concave points_mean, symmetry_mean, fractal_dimension_mean, radius_se, texture_se, perim

In [7]:
df = spark.read.parquet(f'{parquet_path}/Cancer_data.parquet')
df.show(10)

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|
+--------+---------+-----------+------

In [8]:
df.groupBy('diagnosis').count().show()

+---------+-----+
|diagnosis|count|
+---------+-----+
|        B|  357|
|        M|  212|
+---------+-----+



In [11]:
df_agg = (df
          .groupby('diagnosis')
          .agg(F.count('radius_mean').alias('n'),
               F.mean('radius_mean').alias('avg'),
               F.min('radius_mean').alias('min'),
               F.max('radius_mean').alias('max')
               ))
df_agg.show(10)

+---------+---+-----------------+-----+-----+
|diagnosis|  n|              avg|  min|  max|
+---------+---+-----------------+-----+-----+
|        B|357|12.14652380952381|10.03|9.904|
|        M|212|17.46283018867925|10.95|28.11|
+---------+---+-----------------+-----+-----+



In [13]:
df_pivot = (df
            .withColumn('Bigger',col('radius_mean')>15)
            .groupBy('Bigger')
            .pivot('diagnosis')
            .count()
)
df_pivot.show()

+------+---+---+
|Bigger|  B|  M|
+------+---+---+
|  true|  6|135|
| false|351| 77|
+------+---+---+



In [14]:
df_sort = df.orderBy('radius_mean')
df_sort.show(3)

+------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
|    id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|
+------+---------+-----------+------------

In [15]:
df_sort = df.orderBy(F.desc('radius_mean'))
df_sort.show(3)

+------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
|    id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|
+------+---------+-----------+------------

In [25]:
df1 = df.select('id','diagnosis','perimeter_mean').limit(1000)
df2 = df.select('id','area_mean')

In [26]:
df_join = df1.join(df2,'id','left')
df_join.show(3)

+--------+---------+--------------+---------+
|      id|diagnosis|perimeter_mean|area_mean|
+--------+---------+--------------+---------+
|  842302|        M|         122.8|     1001|
|  842517|        M|         132.9|     1326|
|84300903|        M|           130|     1203|
+--------+---------+--------------+---------+
only showing top 3 rows



In [29]:
df_join_2 = df1.join(df2,'id','outer')
df_join_2.show(2)

+------+---------+--------------+---------+
|    id|diagnosis|perimeter_mean|area_mean|
+------+---------+--------------+---------+
|842302|        M|         122.8|     1001|
|842517|        M|         132.9|     1326|
+------+---------+--------------+---------+
only showing top 2 rows



In [31]:
df_join_3 = df1.join(df2,'id','inner')
df_join_3.show(2)

+------+---------+--------------+---------+
|    id|diagnosis|perimeter_mean|area_mean|
+------+---------+--------------+---------+
|842302|        M|         122.8|     1001|
|842517|        M|         132.9|     1326|
+------+---------+--------------+---------+
only showing top 2 rows



In [32]:
df_join_4 = df1.join(df2,'id','right')
df_join_4.show(2)

+------+---------+--------------+---------+
|    id|diagnosis|perimeter_mean|area_mean|
+------+---------+--------------+---------+
|842302|        M|         122.8|     1001|
|842517|        M|         132.9|     1326|
+------+---------+--------------+---------+
only showing top 2 rows



In [40]:
df
for i in range(100):
    df.count()

In [41]:
df.cache()
for i in range(100):
    df.count()

In [42]:
df.unpersist()

DataFrame[id: string, diagnosis: string, radius_mean: string, texture_mean: string, perimeter_mean: string, area_mean: string, smoothness_mean: string, compactness_mean: string, concavity_mean: string, concave points_mean: string, symmetry_mean: string, fractal_dimension_mean: string, radius_se: string, texture_se: string, perimeter_se: string, area_se: string, smoothness_se: string, compactness_se: string, concavity_se: string, concave points_se: string, symmetry_se: string, fractal_dimension_se: string, radius_worst: string, texture_worst: string, perimeter_worst: string, area_worst: string, smoothness_worst: string, compactness_worst: string, concavity_worst: string, concave points_worst: string, symmetry_worst: string, fractal_dimension_worst: string, _c32: string]

In [45]:
df_part = df.repartition(10)
df_part.rdd.getNumPartitions()

10

1 - No VSCode ou Google Colab resolva os exercícios usando PySpark:
 - Abra o arquivo realtor-data.csv
 - Salve e leia novamente o arquivo no formato parquet
 - Gere um cache da tabela com 32 partições 
 - Crie uma pivot table da contagem de state e status.

In [46]:
df_realtor = (spark.read.format('csv').option('header','true').load('../datasets/realtor-data.csv'))
df_realtor.show(2)

+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
|  status|bed|bath|acre_lot|    city|      state|zip_code|house_size|prev_sold_date|   price|
+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
|for_sale|3.0| 2.0|    0.12|Adjuntas|Puerto Rico|   601.0|     920.0|          NULL|105000.0|
|for_sale|4.0| 2.0|    0.08|Adjuntas|Puerto Rico|   601.0|    1527.0|          NULL| 80000.0|
+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
only showing top 2 rows



In [47]:
df_realtor.write.parquet('../datasets/parquets/realtor_data.parquet')

In [50]:
df_realtor = spark.read.parquet('../datasets/parquets/realtor_data.parquet')
df_realtor.show(2)

+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
|  status|bed|bath|acre_lot|    city|      state|zip_code|house_size|prev_sold_date|   price|
+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
|for_sale|3.0| 2.0|    0.12|Adjuntas|Puerto Rico|   601.0|     920.0|          NULL|105000.0|
|for_sale|4.0| 2.0|    0.08|Adjuntas|Puerto Rico|   601.0|    1527.0|          NULL| 80000.0|
+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
only showing top 2 rows



In [51]:
df_realtor.cache()

DataFrame[status: string, bed: string, bath: string, acre_lot: string, city: string, state: string, zip_code: string, house_size: string, prev_sold_date: string, price: string]

In [54]:
df_realtor = df_realtor.repartition(32)
df_realtor.rdd.getNumPartitions()

32

In [57]:
df_realtor.groupBy('state').pivot('status').count().show()

+--------------+--------+--------------+
|         state|for_sale|ready_to_build|
+--------------+--------+--------------+
|   Connecticut|   12178|          NULL|
|       Vermont|    1324|          NULL|
|   Puerto Rico|   24679|          NULL|
|Virgin Islands|    2573|          NULL|
|  Rhode Island|    2401|          NULL|
|      Virginia|       3|          NULL|
|    New Jersey|       2|          NULL|
| Massachusetts|   52513|           181|
| New Hampshire|    2232|          NULL|
|     Tennessee|      16|          NULL|
|South Carolina|      24|          NULL|
|      New York|    1874|          NULL|
+--------------+--------+--------------+



Mostre as 5 cidades que possuem as maiores casas

In [62]:
(df_realtor
.fillna(0,subset = 'house_size')
.groupBy('city')
.agg(F.max('house_size').alias('house_size_max'))
.orderBy(F.desc('house_size_max'))
.show(5))

+----------+--------------+
|      city|house_size_max|
+----------+--------------+
|     Acton|       99999.0|
|Middletown|       99999.0|
|  Chicopee|        9999.0|
|   Holyoke|        9999.0|
|    Dorado|         999.0|
+----------+--------------+
only showing top 5 rows



Crie a seguinte tabela, realize um inner join com essa tabela. Conte o número de registros do resultado final.

In [60]:
import pandas as pd
df = pd.DataFrame({'state':['Connecticut',
                            'Puerto Rico',
                            'Massachusetts',
                            'New York'],
                    'population':[3606,3264,6985,8468]})
df

,state,population
0,Connecticut,3606
1,Puerto Rico,3264
2,Massachusetts,6985
3,New York,8468


In [63]:
df_sk = spark.createDataFrame(df)
df_sk.show()

+-------------+----------+
|        state|population|
+-------------+----------+
|  Connecticut|      3606|
|  Puerto Rico|      3264|
|Massachusetts|      6985|
|     New York|      8468|
+-------------+----------+



In [69]:
df_realtor.join(df_sk,'state','inner').orderBy('population').count()

91425